In [ ]:
# Project to analyze relationship between development team estimates in abstract points (story points) and real calendar and working days.

from jira import JIRA
from collections import OrderedDict
import pandas as pd
import numpy as np
import dateutil.parser
from datetime import date
import plotly.graph_objects as go
import math
from IPython.core.display import HTML


jira = JIRA(basic_auth=('jira_user', 'jira_password'), options={'server': 'jira_server_url'})

dev_statuses = ['Development: In progress']

default_reversed_search_value = True


def get_issue_sp(issue):
    return issue.fields.customfield_11212


def get_issue_key(issue):
    return issue.key


def get_issue_title(issue):
    return str(issue.fields.summary)


def get_issue_resolution_date_str(issue):
    return str(issue.fields.resolutiondate)


def get_issue_to_status_date(issue, status, reversed_search=default_reversed_search_value):
    histories = reversed(issue.changelog.histories) if reversed_search else issue.changelog.histories
    for history in histories:
        for item in history.items:
            if item.field == 'status':
                if item.toString.lower() == str(status).lower():
                    return history.created
    
    return None


def get_issue_from_status_date(issue, status, reversed_search=default_reversed_search_value):
    histories = reversed(issue.changelog.histories) if reversed_search else issue.changelog.histories
    for history in histories:
        for item in history.items:
            if item.field == 'status':
                if item.fromString.lower() == str(status).lower():
                    return history.created
    
    return None


def get_duedate_move_info(issue):
    tracked_status = 'Developement: In Progress'
    if get_issue_to_status_date(issue, tracked_status, reversed_search=False) is None:
        return None
    issue_dev_start = dateutil.parser.parse(get_issue_to_status_date(issue, tracked_status, reversed_search=False)).replace(tzinfo=None)
    issue_dev_end = dateutil.parser.parse(get_issue_from_status_date(issue, tracked_status, reversed_search=True)).replace(tzinfo=None)

    histories = issue.changelog.histories
    move_dates = []
    move_count = 0
    
    for history in histories:
        for item in history.items:
            if item.field == 'duedate':
                if item.fromString is not None:
                    event_date = dateutil.parser.parse(history.created).replace(tzinfo=None)
                    if issue_dev_start.date() < event_date.date() < issue_dev_end.date():
                        move_count += 1 
                        move_dates.append(history.created[:16])
                    
    return dict(move_count=move_count, move_dates=move_dates)


def get_first_duedate_details(issue):
    tracked_status = 'Developement: In Progress'
    if get_issue_to_status_date(issue, tracked_status, reversed_search=False) is None:
        return None
    issue_dev_start = dateutil.parser.parse(get_issue_to_status_date(issue, tracked_status, reversed_search=False)).replace(tzinfo=None)
    issue_dev_end = dateutil.parser.parse(get_issue_from_status_date(issue, tracked_status, reversed_search=True)).replace(tzinfo=None)
    
    for history in issue.changelog.histories:
        for item in history.items:
            if item.field == 'duedate' and item.toString is not None:
                duedate = dateutil.parser.parse(item.toString).replace(tzinfo=None)
                if duedate > issue_dev_start:
                    abs_duedate_error_cd = (issue_dev_end.date() - duedate.date()).days
                    rel_duedate_error_cd = int(round((abs_duedate_error_cd / (duedate.date() - issue_dev_start.date()).days), 2) * 100)
                    
                    return {'duedate':duedate, 
                            'issue_dev_start':issue_dev_start,
                            'issue_dev_end':issue_dev_end,
                            'abs_dd_err':abs_duedate_error_cd,
                            'rel_dd_err':rel_duedate_error_cd
                           }
    return None


def get_issue_property_change_stats(issue, property_name, property_value, 
                                    left_limit_date=None, right_limit_date=None):
    
    prop_change_dates_list = []
    prop_change_dates = {}
    workdays_total = 0
    caldays_total = 0
    property_found = False
    
    ###LOOP START###
    for history in issue.changelog.histories:
        for item in history.items:
            if item.field == property_name:
                if item.toString is not None and item.toString.lower() == property_value.lower():
                    prop_change_dates['begin_date'] = dateutil.parser.parse(history.created).replace(tzinfo=None)
                if item.fromString is not None and item.fromString.lower() == property_value.lower():
                    prop_change_dates['end_date'] = dateutil.parser.parse(history.created).replace(tzinfo=None)
        
        if 'end_date' in prop_change_dates and 'begin_date' in prop_change_dates:
            if left_limit_date is not None and right_limit_date is not None:
                if prop_change_dates['begin_date'].date() < left_limit_date.date() or prop_change_dates['end_date'].date() > right_limit_date.date():
                    prop_change_dates = {}
                    continue
            property_found = True
            workdays_count = np.busday_count(prop_change_dates['begin_date'].date(), prop_change_dates['end_date'].date())
            caldays_count = round((prop_change_dates['end_date'] - prop_change_dates['begin_date']).total_seconds()/60/60/24, 2)
            workdays_total += workdays_count
            caldays_total += caldays_count
            prop_change_dates['workdays_count'] = workdays_count
            prop_change_dates['caldays_count'] = caldays_count             
            prop_change_dates_list.append(prop_change_dates)            
            prop_change_dates = {} 
    ###LOOP END###
    
    if not property_found:
        return None
        
    return {"workdays_total": workdays_total, "caldays_total": caldays_total, "transitions": prop_change_dates_list}


def get_issue_status_change_stats(issue, status):
    return get_issue_property_change_stats(issue, 'status', status)
    
    
def get_issue_blocking_stats(issue):
    block_search_status = 'Developement: In Progress'
    if get_issue_to_status_date(issue, block_search_status, reversed_search=False) is None:
        return None
    block_search_startdate = dateutil.parser.parse(get_issue_to_status_date(issue, block_search_status, reversed_search=False)).replace(tzinfo=None)
    block_search_enddate = dateutil.parser.parse(get_issue_from_status_date(issue, block_search_status, reversed_search=True)).replace(tzinfo=None)
    
    return get_issue_property_change_stats(issue, 'Flagged', 'Impediment',
                                    left_limit_date=block_search_startdate, right_limit_date=block_search_enddate)


def get_portfolio_details(portfolio):
    
    total_caldays = 0
    total_workdays = 0
    blocking_caldays = 0
    blocking_workdays = 0
    
    for status in dev_statuses:
        status_change_stats = get_issue_status_change_stats(portfolio, status)
        if status_change_stats:
            total_caldays += status_change_stats['caldays_total'] 
            total_workdays += status_change_stats['workdays_total']
    
    issue_blocking_stats = get_issue_blocking_stats(portfolio)
    if issue_blocking_stats:
        blocking_caldays = issue_blocking_stats['caldays_total']
        blocking_workdays = issue_blocking_stats['workdays_total']
    
    caldays_wo_block = total_caldays - blocking_caldays
    workdays_wo_block = total_workdays - blocking_workdays
    
    sp = get_issue_sp(portfolio)
    
    dd_move_qty = None
    dev_start = None
    dev_end = None
    first_duedate = None
    abs_dd_err = None
    rel_dd_err = None
    if get_first_duedate_details(portfolio) is not None:
        dd_move_qty = get_duedate_move_info(portfolio)['move_count']
        dev_start = get_first_duedate_details(portfolio)['issue_dev_start']
        dev_end = get_first_duedate_details(portfolio)['issue_dev_end']
        first_duedate = get_first_duedate_details(portfolio)['duedate']
        abs_dd_err = get_first_duedate_details(portfolio)['abs_dd_err']
        rel_dd_err = get_first_duedate_details(portfolio)['rel_dd_err']
        
    to_os_date = get_issue_to_status_date(portfolio, "Feedback")
            
    workdays_wo_block_in_one_sp = round(workdays_wo_block / float(sp), 2) if sp is not None and sp > 0 else None
    caldays_wo_block_in_one_sp = round(caldays_wo_block / float(sp), 2) if sp is not None and sp > 0 else None
    total_workdays_in_one_sp = round(total_workdays / float(sp), 2) if sp is not None and sp > 0 else None
    total_caldays_in_one_sp = round(total_caldays / float(sp), 2) if sp is not None and sp > 0 else None    
    
    return OrderedDict(
        title = get_issue_title(portfolio),
        key = get_issue_key(portfolio),
        dev_start = dev_start,
        dev_end = dev_end,
        first_duedate = first_duedate,
        abs_dd_err = abs_dd_err,
        rel_dd_err = rel_dd_err,
        dd_move_qty = dd_move_qty,
        resolution_date = to_os_date if to_os_date is not None else get_issue_resolution_date_str(portfolio),
        total_wrkdays = total_workdays,
        total_cldays = total_caldays,
        block_wrkdays = blocking_workdays,
        block_cldays = blocking_caldays,
        wrkdays_wo_block = workdays_wo_block,
        cldays_wo_block = caldays_wo_block,
        sp = sp,
        wdays_wo_block_in_sp = workdays_wo_block_in_one_sp,
        cldays_wo_block_in_sp = caldays_wo_block_in_one_sp,
        ttl_wdays_in_sp = total_workdays_in_one_sp,
        ttl_cldays_in_sp = total_caldays_in_one_sp
    )


def get_portfolios_stats(dev_team, begin_date, end_date):

    JQL_QUERY = 'project = "Development" and type != Epic and "Development Team" = "{}" and resolutiondate >= {} and resolutiondate <= {} and resolution not in ("Won\'t Fix", Duplicate, "Hold On", "Not a bug")'.format(dev_team, begin_date, end_date)

    portfolios = jira.search_issues(JQL_QUERY, expand='changelog', maxResults=1000)

    portfolio_stats_list = []
    for portfolio in portfolios:
        portfolio_stats_list.append(get_portfolio_details(portfolio))

    return portfolio_stats_list


def display_scatter(title, xaxis_title, yaxis_title, data_categories, visible_category, x_data, y_data_df_name, 
                    scatter_mode='markers', line_props=None, marker_size=10):
    
    fig = go.Figure()
    
    for data_category in data_categories:
        visible_value = True if data_category == visible_category else 'legendonly'
        fig.add_trace(go.Scatter(x=x_data,
                                y=y_data_df_name[data_category].tolist(),
                                mode=scatter_mode,
                                marker_size=marker_size,
                                line = line_props,
                                text=y_data_df_name['title'],
                                name = data_category, 
                                visible = visible_value,
                                hovertemplate=
                                '<b>text</b>: %{text}<br>' +
                                '<b>x</b>: %{x}<br>'+
                                '<b>y</b>: %{y}'+
                                "<extra></extra>")
                        )
        
    fig.update_layout(title=title, xaxis_title=xaxis_title, yaxis_title=yaxis_title)
    fig.show()


def show_sp_stats(dev_team, begin_date, end_date, excluded_portfolios=[]):
    
    ### Get data from portfolios and create pandas dataframes
    stats = get_portfolios_stats(dev_team, begin_date, end_date)
    full_data = pd.DataFrame(stats)
    polished_data = full_data[~full_data.key.isin(excluded_portfolios)]
    polished_data = polished_data[polished_data.ttl_cldays_in_sp.notnull()]
    full_data['resolution_date'] = pd.to_datetime(full_data['resolution_date'], format="%Y-%m-%d %H:%M").dt.tz_localize(None)
    polished_data['resolution_date'] = pd.to_datetime(polished_data['resolution_date'], format="%Y-%m-%d %H:%M").dt.tz_localize(None)
    polished_data = polished_data[polished_data['resolution_date'] >= dateutil.parser.parse(begin_date)]
    
    ### Display pandas dataframe with full data from portfolios and short stats about portfolios quantity
    with pd.option_context('display.max_rows', None, 'display.max_columns', None):  
        display(full_data.sort_values(by='ttl_cldays_in_sp', ascending=False).style.format({'key': __make_clickable__}))
    print('Initial number of tasks: {} | Number of excluded tasks: {} | Number of tasks chosen for analysis: {}'.format(len(full_data), len(full_data) - len(polished_data), len(polished_data)))
    if (len(full_data) - len(polished_data) > 0):
        print('\nExcluded tasks')
        display(full_data[~full_data.isin(polished_data)].dropna(how = 'all'))

    
    ### Prepare data and display dataframe with stats on average sp size in dev days
    dev_days_types = ['wrkdays_wo_block', 'cldays_wo_block', 'total_wrkdays', 'total_cldays']
    avg_sp_days_data = OrderedDict()
    for dev_days_type in dev_days_types:
        avg_sp_days_data[dev_days_type] = [round(polished_data[dev_days_type].sum() / polished_data.sp.sum(), 2)]
    display(HTML('<br><h4>Average SP size in days</h4>'))
    display(pd.DataFrame(avg_sp_days_data).transpose())
    
    ### Prepare data and display dataframe with stats on sp size in dev days by procentiles
    percentile_scale = [50, 75, 85, 100]
    percentile_scale_labels = ["{}%".format(prc) for prc in percentile_scale]
    sp_size_types = ['wdays_wo_block_in_sp', 'cldays_wo_block_in_sp', 'ttl_wdays_in_sp', 'ttl_cldays_in_sp']
    sp_days_procentiles_data = OrderedDict()
    sp_days_procentiles_data['procentile'] = percentile_scale_labels
    for sp_size_type in sp_size_types:
        sp_days_procentiles_data[sp_size_type] = [round(np.percentile(sorted(polished_data[sp_size_type].tolist()), prc), 2) for prc in percentile_scale]
    display(HTML('<br><h4>SP size in days by procentiles</h4>'))
    display(pd.DataFrame(sp_days_procentiles_data))
    
    ### Display scatter plot of relationship btw portfolio size in sp and portfolio dev days
    display_scatter(title='"SP - portfolio dev days" relationship', 
                    xaxis_title='Portfolio size, SP', 
                    yaxis_title='Portfolio dev days', 
                    data_categories=dev_days_types, 
                    visible_category='wrkdays_wo_block',
                    x_data=polished_data.sp.tolist(), 
                    y_data_df_name=polished_data)
    
    ### Display scatter plot of relationship btw portfolio size in sp and days in 1sp
    display_scatter(title='"SP - #of dev days in 1SP" relationship',
                    xaxis_title='Portfolio size, SP',
                    yaxis_title='Number of dev days in 1 sp', 
                    data_categories=sp_size_types, 
                    visible_category='wdays_wo_block_in_sp',
                    x_data=polished_data.sp.tolist(), 
                    y_data_df_name=polished_data)
    
    ### Display scatterplot of dev days in 1 sp dynamics
    data_by_date = polished_data.sort_values(by='resolution_date')
    display_scatter(title='Dev days in 1 sp dynamics',
                    xaxis_title='Portfolio resolution date',
                    yaxis_title='Dev days in 1 sp', 
                    data_categories=sp_size_types, 
                    visible_category='wdays_wo_block_in_sp',
                    x_data=data_by_date.resolution_date.tolist(), 
                    y_data_df_name=data_by_date,
                    scatter_mode='lines+markers',
                    line_props=dict(width=2, dash='dot'),
                    marker_size=9)
    
    ### Display scatter plot of relationship btw portfolio size in sp and duedate moves qty
    display_scatter(title='"SP - duedate moves" relationship', 
                    xaxis_title='Portfolio size, SP', 
                    yaxis_title='Portfolio due date moves qty', 
                    data_categories=['dd_move_qty'], 
                    visible_category='dd_move_qty',
                    x_data=polished_data.sp.tolist(), 
                    y_data_df_name=polished_data)
    
    ### Display scatter plot of relationship btw portfolio size in sp and duedate error in days
    ### Duedate error is calculated as (dev_end_date - first duedate)
    display_scatter(title='"SP - duedate error" relationship', 
                    xaxis_title='Portfolio size, SP', 
                    yaxis_title='Portfolio duedate error, сdays and %', 
                    data_categories=['abs_dd_err', 'rel_dd_err'], 
                    visible_category='abs_dd_err',
                    x_data=polished_data.sp.tolist(), 
                    y_data_df_name=polished_data)
    
    
def __make_clickable__(val):
        return '<a href="{}" target="_blank">{}</a>'.format("https://jira.hh.ru/browse/"+val, val)


In [ ]:
today = date.today().strftime("%Y-%m-%d")

dev_team = 'Clickme'
begin_date = '2019-06-01'
end_date = today
show_sp_stats(dev_team, begin_date, end_date, excluded_portfolios=[])

In [ ]:
import datetime

def get_shifted_date(start_date, days_offset, use_workdays=True):
    day_increment = datetime.timedelta(1)
    calculated_date = dateutil.parser.parse(start_date)
    count = 0
    while count < days_offset:
        calculated_date += day_increment
        if use_workdays:
            if calculated_date.weekday() <= 4:
                count += 1
        else:
            count += 1
    return calculated_date


### Fill parameters below to calculate duedate

dev_start_date      = '2019-11-25'    ### use format "yyyy-mm-dd"
portfolio_size_sp   = 13              ### portfolio size in sp
days_in_sp          = 2.82            ### size of sp in days
use_workdays        = True            ### use True in case of workdays, or False - if calend days


num_of_days = math.ceil(portfolio_size_sp * days_in_sp)
print("Forecast to complete project:", get_shifted_date(dev_start_date, num_of_days, use_workdays).strftime("%d.%m.%Y"))
print("Number of days:", num_of_days)